In [1]:
# This document details the circuit design decisions made for the LTC7871 buck converter. The goal here is to provide an easy
# way to evaluate how component values will affect the overall behavior and performance, without having to restrict myself to
# an excel sheet or a SPICE simulation. This is a living document and will be updated as I learn more about the LTC7871 and
# develop the hardware.

import math
from enum import Enum

In [ ]:
# *****************************************************************************
# System Design Constraints
#   Design solar cell is similar to HiS-S400YH(BK), 400W, 45.3V OC, 20% efficiency, 37.7V 10.61A MPP.
#   Approximate configuration is 2SnP, where n:
#       1: 10.61A, 800W (75.4v)
#       2: 21.22A, 1600W
#       3: 31.83A, 2400W
#       4: 42.44A, 3200W
#       5: 53.05A, 4000W
#       6: 63.66A, 4800W
#       7: 74.27A, 5600W
#       8: 84.88A, 6400W
#       9: 95.49A, 7200W
#      10: 106.1A, 8000W
#      11: 116.71A, 8800W
#      12: 127.32A, 9600W
#      13: 137.93A, 10400W
# *****************************************************************************
# Overall System Power
MPPT_Pwr_Avg = 10000.0  # Average output power (watts)
MPPT_Pwr_Max = 13500.0  # Maximum output power (watts)

# Photovoltaic (PV) Panel Limits
PV_V_In_Max = 90.0  # Maximum open circuit input voltage (volts)
PV_V_In_Min = 18.0  # Minimum open circuit input voltage (volts)
PV_I_In_Max = 150.0  # Maximum input current (amps)

# Buck Converter Design (Single Unit)
Pwr_max = 2500.0 # Maximum power output of the converter in watts.
Eff_min = 0.75 # Estimated minimum efficiency of the converter under typical conditions.
Eff_max = 0.97 # Estimated maximum efficiency of the converter under typical conditions.
Vout_min = 12.0 # Minimum output voltage. Not enough to charge, but could power an inverter?
Vout_max = 56.0 # High enough to charge a 48V battery bank.
Vin_min = 24.0 # Photovoltaic input voltage required to charge a 12v battery bank with some headroom.
Vin_max = 90.0 # Photovoltaic input voltage max of the LTC7871 with some headroom.
Iout_max = Pwr_max / Vout_min # Maximum output current in amps.
Iout_min = Pwr_max / Vout_max # Minimum output current in amps.

In [54]:
# *****************************************************************************
# Operating Conditions to Evaluate
# *****************************************************************************
# TODO: Create a table of evaluation conditions and swap them in.

# Configurable Fields

# Number of parallel phase outputs in the design
num_phases = 6

# Output of the converter (V)
VLow = 12.0

# Input of the converter (V)
VHigh = 75.4

# Max output current (A)
op_Iout_max = Pwr_max / VLow
op_Iout_phase = op_Iout_max / num_phases

# Allowed inductor ripple current percentage
IRipplePct = 0.4

# Desired max ripple current in the inductor (A)
dIL = Iout_max * IRipplePct

# Desired max ripple voltage on the output (V)
dVLow_ref = 100e-3

# Core switching frequency (Hz)
Fosc = 650e3

# DRVcc (Pin 46) Gate Driver Supply Voltage.
# Must be either 5v, 8v, or 10v.
DRVcc = 10.0

# Known Limits
assert 60e3 <= Fosc <= 750e3, "Switching frequency must be between 60kHz and 750KHz"

In [55]:
# *****************************************************************************
# Minimum On-Time (pg 29)
# *****************************************************************************
ref_on_time = 150e-9 # (sec) Best case performance from datasheet
act_on_time = VLow / (VHigh * Fosc)
print(f"Minimum On-Time: {ref_on_time * 1e9:.2f}nS, Actual Min: {act_on_time*1e9:.2f}nS")
assert ref_on_time < act_on_time

Minimum On-Time: 150.00nS, Actual Min: 244.85nS


In [56]:
# *****************************************************************************
# Current Limit Programming
# *****************************************************************************
class ILim(Enum):
    """ Enumeration of the current limit programming options for the LTC7871 Table 5 """
    ILIM_0 = 0
    ILIM_1_4 = 1
    ILIM_FLOAT = 2
    ILIM_3_4 = 3
    ILIM_FULL = 4

# Maximum voltage sense values for a DCR type current sense resistor.
# Values are in volts.
VSense_Max_DCR = {
    ILim.ILIM_0: 0.1,
    ILim.ILIM_1_4: 0.2,
    ILim.ILIM_FLOAT: 0.3,
    ILim.ILIM_3_4: 0.4,
    ILim.ILIM_FULL: 0.5
}

VSense_VDiv_Resistors = {
    ILim.ILIM_0: (22e3, 1e3),
    ILim.ILIM_1_4: (10.2e3, 3.4e3),
    ILim.ILIM_FLOAT: (10e3, 10e3),
    ILim.ILIM_3_4: (3.4e3, 10.2e3),
    ILim.ILIM_FULL: (1e3, 22e3)

}

# Selection for the current limit programming.
ILim_Select = ILim.ILIM_0
VSense_Max = VSense_Max_DCR[ILim_Select]

# ILIM resistor values
R1, R2 = VSense_VDiv_Resistors[ILim_Select]
print(f"VSense Max: {VSense_Max:.2f}V")
print(f"Vdiv_R1: {R1 / 1e3:.2f} kOhms")
print(f"Vdiv_R2: {R2 / 1e3:.2f} kOhms")

VSense Max: 0.10V
Vdiv_R1: 22.00 kOhms
Vdiv_R2: 1.00 kOhms


In [57]:
# *****************************************************************************
# DCR and Inductor Selection (Figure 3) *** PER PHASE ***
# *****************************************************************************
R1 = 1e3
C1 = 0.080e-6
R2 = 4e3
C2 = 0.1e-6
L = 1.0e-6
DCR = 2.5e-3

# Ensure VSenseMax isn't exceeded
act_Vsense = DCR * (op_Iout_phase + (dIL / 2.0))
assert act_Vsense < VSense_Max, f"The actual voltage sense {act_Vsense} > maximum allowed voltage sense {VSense_Max}"
print(f"Actual VSense: {act_Vsense:.2f}V")

# Compute approximate equality of the output inductor characteristics and the two filter stages
tolerance = 1e-5
L_by_DCR = round(L / DCR, 6)
R1C1_5 = round(R1 * C1 * 5, 6)
R2C2 = round(R2 * C2, 6)

# Compute the minimum inductor value to achieve desired ripple current. << current == lower core loss
L_min = ((VHigh - VLow)/(Fosc * dIL)) * (VLow / VHigh)
print(f"Inductance: {L * 1e6:.2f}uH, Min: {L_min * 1e6:.2f}uH")
assert L >= L_min, f"Inductor value {L} < minimum inductor value {L_min}"

# Validate relationship between the inductor and the two filter stages
print(f"L/DCR: {L_by_DCR}, R1C1_5: {R1C1_5}, R2C2: {R2C2}")
assert math.isclose(L_by_DCR, R1C1_5, abs_tol=tolerance), f"L/DCR {L_by_DCR} != R1C1_5 {R1C1_5}"
assert math.isclose(L_by_DCR, R2C2, abs_tol=tolerance), f"L/DCR {L_by_DCR} != R2C2 {R2C2}"

# Ensure the current sense input ripple voltage is less than 2mV
dVsense = (VLow / VHigh) * ((VHigh - VLow) / (R1 * C1 * Fosc))
assert dVsense < 10e-3, f"SNSA+, SNS- ripple voltage {dVsense} > 10mV recommended"

# Peak Current Rating
L_IRated = op_Iout_phase + dIL / 2.0
print(f"Inductor Avg Current: {op_Iout_phase:.2f}A")
print(f"Inductor Peak Current: {L_IRated:.2f}A")

# Compute copper loss
L_i2r_loss = op_Iout_phase ** 2 * DCR

# Summarize this block
print(f"Exp SNS +/- Ripple {dVsense * 1e3:.2f}mV")
print(f"Inductor Avg I2R Loss: {L_i2r_loss:.2f}W")

Actual VSense: 0.08V
Inductance: 1.00uH, Min: 0.47uH
L/DCR: 0.0004, R1C1_5: 0.0004, R2C2: 0.0004


AssertionError: SNSA+, SNS- ripple voltage 0.19404203223831867 > 10mV recommended

In [ ]:
# *****************************************************************************
# Mosfet Design *** PER PHASE ***
# *****************************************************************************
# Pg. 24
# Compute Ptop and PBot
# Take care for miller capacitance

# Mosfet Characteristics: MCAC80N10Y
mos_temp_delta = 0.005  # per degree celcius (estimate, unitless)
mos_Rds_on = 3.3e-3  # Resistance when fully on (Ohm)
mos_Crss = 15e-12  # Reverse transfer capacitance aka miller capacitance (Farad)
mos_Iavg = 80.0 # Continuous current (Amps)
mos_Vth = 2.0 # Transition threshold voltage (Volts)
drv_Rdr = 1.5 # Resistance of top switch driver (Ohm), from LTC7060 R_UP(TG)?

# Junction Temp Adjustment
est_junction_temp = 75.0  # Celcius
ambient_temp = 25.0  # Celcius
temp_factor = mos_temp_delta * (est_junction_temp - ambient_temp)

# Top Switch Mosfet Dissipation
mos_top_1 = (VLow / VHigh) * (op_Iout_phase ** 2.0) * (1.0 + temp_factor) * mos_Rds_on
mos_top_2 = (VHigh ** 2.0) * (op_Iout_phase / 2.0) * drv_Rdr * mos_Crss
mos_top_3 = ((1.0 / (DRVcc - mos_Vth)) + (1.0 / mos_Vth)) * Fosc
mos_top_pwr_dissipation = mos_top_1 + mos_top_2 * mos_top_3

# Bottom Switch Mosfet Dissipation
mos_bot_1 = (VHigh - VLow) / VHigh
mos_bot_2 = op_Iout_phase ** 2.0
mos_bot_3 = (1.0 + temp_factor) * mos_Rds_on
mos_bot_pwr_dissipation = mos_bot_1 * mos_bot_2 * mos_bot_3

print(f"Dissipation Top: {mos_top_pwr_dissipation:.2f}W")
print(f"Dissipation Bottom: {mos_bot_pwr_dissipation:.2f}W")

Dissipation Top: 0.35W
Dissipation Bottom: 0.67W


In [ ]:
# *****************************************************************************
# High Side Capacitor Design
# *****************************************************************************
# Pg. 24
# Cin sized by RMS current of a single channel (maybe from dIL?)
Cin_max_Irms = (op_Iout_max / VHigh) * math.sqrt(VLow * (VHigh - VLow))

print(f"Input RMS Current: {Cin_max_Irms:.2f}A")

Input RMS Current: 30.48A


In [ ]:
# *****************************************************************************
# Low Side Capacitor Design
#   I learned from various online sources that I can combine ceramic and
#   electrolytic capacitors to achieve the desired ESR and capacitance. The
#   ceramic capacitors provide low ESR and high frequency performance, while
#   the electrolytic capacitors provide bulk capacitance. Despite lower ripple
#   current ratings on the electrolytics, the ceramic capacitors will handle
#   the bulk of this due to their lower impendance at higher frequencies.
# *****************************************************************************
# Pg. 25
# The selection of Cout at Vout is driven by ESR.
# - Constraint: Compute output voltage ripple
# - Constraint: Cout > 1/(8f*Rsense)

# Capacitor: CGA6P1X7R1N106M250AC
# https://product.tdk.com/en/search/capacitor/ceramic/mlcc/info?part_no=CGA6P1X7R1N106M250AC
CLow_ceramic_cap = 2e-6  # Output Capacitance (Farads) @48V DC bias
CLow_ceramic_ESR = 3e-3  # Estimated worst case output ESR (ohm) @100kHz-1MHz
CLow_ceramic_N = 4  # Number of capacitors in parallel
_ceramic_eff_esr = CLow_ceramic_ESR / CLow_ceramic_N

# Capacitor: 80ZLH220MEFCT810X23
# https://www.digikey.com/en/products/detail/rubycon/80ZLH220MEFCT810X23/3568221
CLow_electrolytic_cap = 220e-6  # Output Capacitance (Farads)
CLow_electrolytic_ESR = 0.11  # Estimated output ESR (ohm)
CLow_electrolytic_N = 4  # Number of capacitors in parallel
_electrolytic_eff_esr = CLow_electrolytic_ESR / CLow_electrolytic_N

# Compute effective capacitance and ESR from both ceramic and electrolytic capacitors
CLow_eff_ESR = 1.0 / ( (1.0 / _ceramic_eff_esr) + (1.0 / _electrolytic_eff_esr) )
CLow_eff = (CLow_ceramic_cap * CLow_ceramic_N) + (CLow_electrolytic_cap * CLow_electrolytic_N)

# Validate steady state output ripple voltage isn't exceeded
dVLow_act = dIL * (CLow_eff_ESR + (1.0 / (8.0 * Fosc * CLow_eff)))
print(f"Output voltage ripple:\t{dVLow_act * 1e3:.2f}mV, Min: {dVLow_ref * 1e3:.2f}mV")
assert dVLow_act <= dVLow_ref

# Validate minimum capacitance is met
CLow_min_ref = 1.0 / (8.0 * Fosc * DCR)
print(f"Output capacitance:\t{CLow_eff * 1e6:.2f}uF, Min: {CLow_min_ref * 1e6:.2f}uF")
assert CLow_eff > CLow_min_ref

# TODO: Compute power dissipation in the capacitors

Output voltage ripple:	36.07mV, Min: 100.00mV
Output capacitance:	888.00uF, Min: 125.00uF


In [90]:
# *****************************************************************************
# VLow Programmable Range (pg. 17)
# *****************************************************************************
NominalVoltageRanges = {
    # VLow: (RA, RB)
    "12v": (10e3, 100e3),
    "24v": (10e3, 150e3),
    "48v": (12e3, 348e3),
}

IDAC_min = -64e-6
IDAC_max = 64e-6
IDAC_steps = 128

for name, (RA, RB) in NominalVoltageRanges.items():
    VLow_Nominal = 1.2 * (1.0 + (RB / RA))
    VLow_programmable_min = 1.2 * (1.0 + (RB / RA)) - (IDAC_max * RB)
    VLow_programmable_max = 1.2 * (1.0 + (RB / RA)) - (IDAC_min * RB)
    VLow_resolution = (VLow_programmable_max - VLow_programmable_min) / IDAC_steps

    print(f"{name} -- Range: {VLow_programmable_min:.2f}V - {VLow_programmable_max:.2f}V, Resolution: {VLow_resolution:.4f}V")


12v -- Range: 6.80V - 19.60V, Resolution: 0.1000V
24v -- Range: 9.60V - 28.80V, Resolution: 0.1500V
48v -- Range: 13.73V - 58.27V, Resolution: 0.3480V
